# Notebook responsável por apresentar os comandos básicos para manipulação de dados no mongo

## Crud com apenas vários documentos:
- Inserir vários documentos **compostos**
- Recuperar vários documentos **compostos**
- Atualizar vários documentos **compostos**
- Excluir vários documentos **compostos**

## Bibliotecas importantes

In [ ]:
from pymongo import MongoClient
import pprint
import numpy as np
import pandas as pd

# para geração de dados fake
from faker import Faker


## Funções uteis

In [ ]:
def criar_endereco():
    return {
        'logradouro':fake.street_name(),
        'numero':fake.building_number(),
        'bairro':fake.bairro(),
        'cidade':fake.city(),
        'uf':fake.administrative_unit(),
        'cep':fake.postcode(),
    }

In [ ]:
def criar_pessoa():
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    
    idade = np.random.randint(18, 50)
    
    pessoa = {
        'cpf': fake.cpf(),
        'nome': f'{primeiro_nome} {ultimo_nome}' ,
        'idade': idade ,
        'data_nascimento':fake.date()
    }        
    pessoa['email']= f'{primeiro_nome.lower()}.{ultimo_nome.lower()}@{fake.free_email_domain()}'
    
    pessoa['endereco'] = criar_endereco()

    pessoa['telefones'] = []
    n = np.random.randint(1, 4)
    for i in range(n):
        pessoa['telefones'].append(fake.msisdn())

    pessoa['formacoes'] = []
    n = np.random.randint(1, 4)
    for i in range(n):
        pessoa['formacoes'].append(fake.job())
        
        
    return pessoa

## Configurações

In [ ]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de vários documentos

### Conectando ao MongoDB

In [ ]:
client = MongoClient('localhost', 27017)
# Se a base de dados não existir, ela é criada automagicamente :-)
db = client['techshot_mongo03']

### Exemplo - Criação de vários documentos compostos

In [ ]:
lista_pessoas = []

for i in range(10):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

In [ ]:
df = pd.DataFrame.from_records(lista_pessoas)
df

In [ ]:
df.info()

### Inserindo os documentos no banco de dados

In [ ]:
# se a coleção não existir, ela é criada automagicamente :-)
colecao_pessoa = db['pessoas']
pessoa_ids = colecao_pessoa.insert_many(lista_pessoas).inserted_ids
pessoa_ids

## Recuperando os documentos inseridos

#### Recuperar todos

In [ ]:
for p in colecao_pessoa.find({}):
    pp.pprint(p)

In [ ]:
type(colecao_pessoa.find({}))

In [ ]:
colecao_pessoa.find({}).explain()

#### Recuperando por algum outro atributo

In [ ]:
df['idade'].mean()

In [ ]:

criterio = {
    'idade': {'$gt':df['idade'].mean()}
}
pp.pprint(criterio)


In [ ]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

In [ ]:
for p in colecao_pessoa.find(criterio):
    pp.pprint(p)

#### Recuperando por algum outro atributo

In [ ]:
df.loc[5, 'endereco']['cidade']

In [ ]:
criterio = {
    'endereco.cidade': {'$eq':df.loc[5, 'endereco']['cidade']}
}
pp.pprint(criterio)


In [ ]:
for p in colecao_pessoa.find(criterio):
    pp.pprint(p)

## Atualizar o documento

In [ ]:
df.loc[3, 'cpf']

In [ ]:
criterio = {
    "cpf":df.loc[3, 'cpf']}

novo_valor = {
    "$set": { "endereco.bairro": "Pampulha" }
}
pp.pprint(novo_valor)


In [ ]:
r = colecao_pessoa.update_many(criterio, novo_valor)
r

In [ ]:
# ref.: https://pymongo.readthedocs.io/en/stable/api/pymongo/results.html
r.matched_count

In [ ]:
r.modified_count

In [ ]:
r.raw_result

In [ ]:
r.upserted_id

In [ ]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

## Remover todos os documentos

In [ ]:
criterio = {}
pp.pprint(criterio)

In [ ]:
r = colecao_pessoa.delete_many(criterio)

In [ ]:
r.raw_result

In [ ]:
p = colecao_pessoa.find_one({})
pp.pprint(p)